In [56]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pymysql 

In [57]:
# 用pymysql连接尝试
# 连接数据库
conn = pymysql.connect(
	host = '123.60.11.177',
	port = 3306,
	user = 'root',
	password = 'ncu@jw114',
	db = 'examArrange1',
	charset='utf8'
	)
cur = conn.cursor()
cur.execute("SELECT VERSION()")
data = cur.fetchall()
cur.execute("SELECT * FROM classroom_inf")
classroom_inf=pd.DataFrame(cur.fetchall())
cur.execute("SELECT * FROM classs_inf")
class_inf=pd.DataFrame(cur.fetchall())
cur.execute("SELECT * FROM teacher_inf")
teacher_inf=pd.DataFrame(cur.fetchall())
cur.execute("SELECT * FROM mid_testinfo")
mid_testinfo=pd.DataFrame(cur.fetchall())
cur.execute("SELECT * FROM dm_classroom_state")
dm_classroom_state=pd.DataFrame(cur.fetchall())
#关闭数据库连接
conn.close()


# 选出公共课

In [58]:
#放回当前学科当前班级对应的班级编号
#输入为一个temp（学姐设计的第一个表）series --与apply使用输入为dataframe
#输出对应的课的班级编号
def get_ci_nation_no(row):
    x=class_inf[0][(class_inf[1]==row[0])&(class_inf[5]==row[2])]
    return x.values[0]
#输入cl_notice_no每班级编号，为int
#输出为当前班级对于的人数
def get_class_student_number(cl_notice_no):
    return class_inf[class_inf[0]==cl_notice_no][6].values

In [83]:
#教室时间对
D=dm_classroom_state[dm_classroom_state[1]!='虚拟教室01'] #维护的一个笛卡儿排课表
dm_classroom_state[dm_classroom_state[1]!='虚拟教室01']
new_D=D[:]
#课表时间对
temp=pd.read_csv("./temp.csv")
ci_nation_no=temp.apply(get_ci_nation_no,axis=1).values
temp.insert(0,"ci_nation_no",ci_nation_no)

# 两个输入数据

In [60]:
new_D #0是序号--后面可以用来存班级编号，
      #1位教室名称，2对应校区，3对应考试时间，4对应的是否能够使用，5对应是否安排考场

,0,1,2,3,4,5
0,1,慧源楼101,前湖校区,1,1,None
1,2,慧源楼101,前湖校区,2,1,None
2,3,慧源楼101,前湖校区,3,1,None
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [61]:
temp #ci_nation_no班级编号，ci_course_no	课程名称（可不用），
     #test_time需要安排的考试时间，ci_class_name班级名称（可不用）

,ci_nation_no,ci_course_no,test_time,ci_class_name
0,202020212000346,104GT002,0,设计学类[205-8]班
1,202020212000347,104GT002,0,设计学类[201-4]班
2,202020212000557,104GT002,0,绘画[201-4]班
3,202020212000863,104GT002,0,管理科学与工程类[201-4]班
4,202020212001028,104GT002,0,新闻传播学类[201-3]班
...,...,...,...,...
289,202020212008325,780GL005,3,基础医学201班
290,202020212008885,780GL005,3,医学影像学201班
291,202020212008886,780GL005,3,医学影像学202班
292,202020212009052,780GL005,3,眼视光医学201班


# 主要函数内容

In [62]:
#功能描述：判断当前考场是否安排
#输入：教室名称，教室时间（--索引当前教室的状态）
#输出：当前教室是否安排，安排则为true
def classroom_isbz(cr_name,time_no):
    return (D[(D[1]==cr_name)&(D[3]==time_no)][5]==0)
#功能描述：判断当前时段的教室是否符合当前的课程，矛盾检测
#输入：教室名称（--求教室的人数），当前班级的人数
#输出：是否能够安排，能安排则为true，不能安排则
def student_num_fit(cr_name,cl_num):
    if (classroom_inf[classroom_inf[1]==cr_name][9].values>=(cl_num-10)):
        return True
    else:
        return False

In [65]:
#功能描述：进行排考
#输入：教室时间笛卡儿乘积；班级编号与时间笛卡儿乘积
#输出：教室时间班级编号笛卡儿
def schedule_algorithm(new_D,temp):
    cannt_find_room_notice=[]
    for cl_no in temp.iloc[:,0].values:
        cl_num=get_class_student_number(cl_no)          #取出对应教室的学生数量
        fit_time=temp[temp['ci_nation_no']==cl_no]["test_time"].values[0] #取出第一个时间段
        fittime_room=D[D[3]==fit_time+1]                           #根据时间段找教室
        for room_name in fittime_room[1].values.tolist():          #遍历教室
            if classroom_isbz(room_name,fit_time).values==False and student_num_fit(room_name,cl_num):
                new_D[(new_D[1]==room_name)&(new_D[3]==fit_time)]=new_D[(new_D[1]==room_name)&(new_D[3]==fit_time)].fillna(0)
                new_D.iloc[new_D[(new_D[1]==room_name)&(new_D[3]==fit_time)].index[0],0]=cl_no
                break
        else:
            cannt_find_room_notice.append(cl_no)
    return new_D,cannt_find_room_notice

In [66]:
#alldata---所有考场的信息
#cannt_find_room---未安排的班级编号
#outdata-----已经安排好的考场情况
alldata,cannt_find_room=schedule_algorithm(new_D,temp) 
outdata=alldata[alldata[5]==0]

In [69]:
alldata

,0,1,2,3,4,5
0,202020212003601,慧源楼101,前湖校区,1,1,0
1,202020212000466,慧源楼101,前湖校区,2,1,0
2,202020212001472,慧源楼101,前湖校区,3,1,0
3,4,慧源楼101,前湖校区,4,1,None
4,5,慧源楼101,前湖校区,5,1,None
...,...,...,...,...,...,...
2419,2420,慧源楼520,前湖校区,20,1,None
2420,2421,慧源楼520,前湖校区,21,1,None
2421,2422,慧源楼520,前湖校区,22,1,None
2422,2423,慧源楼520,前湖校区,23,1,None


In [72]:
print(cannt_find_room[:5])
len(cannt_find_room)

[202020212000346, 202020212000347, 202020212000557, 202020212000863, 202020212001028]


227

In [68]:
outdata

,0,1,2,3,4,5
0,202020212003601,慧源楼101,前湖校区,1,1,0
1,202020212000466,慧源楼101,前湖校区,2,1,0
2,202020212001472,慧源楼101,前湖校区,3,1,0
24,202020212036197,慧源楼111,前湖校区,1,1,0
25,202020212003773,慧源楼111,前湖校区,2,1,0
...,...,...,...,...,...,...
1394,202020212008886,慧源楼327,前湖校区,3,1,0
1417,202020212009053,慧源楼328,前湖校区,2,1,0
1418,202020212009052,慧源楼328,前湖校区,3,1,0
1657,202020212009256,慧源楼340,前湖校区,2,1,0


# 初始数据分析--与前面算法没直接关系

In [67]:
classroom_inf.iloc[:,9].value_counts()    #----考场座位分布情况

38     37
30     36
36      8
58      6
62      5
80      3
48      3
100     1
94      1
68      1
Name: 9, dtype: int64